# Úvod do PyTorch



In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import pickle
from IPython.core.debugger import set_trace

plt.rcParams['figure.figsize'] = (12., 8.)
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

CUDA = True

In [ ]:
import torch

In [ ]:
x = torch.Tensor(5, 3)
x

In [ ]:
x = torch.rand(5, 3)
x

In [ ]:
x.size()

In [ ]:
y = torch.rand(5, 3)
x + y

Konverze PyTorch --> numpy

In [ ]:
x.numpy()

Konverze numpy --> PyTorch

In [ ]:
torch.from_numpy(x.numpy())

## Autograd

In [ ]:
from torch.autograd import Variable

In [ ]:
x = torch.rand(5, 3)
x = Variable(x, requires_grad=True)
x

In [ ]:
x + 2

In [ ]:
y = torch.rand(5, 3)
y = Variable(y, requires_grad=False)
y

In [ ]:
z = x + y
z

In [ ]:
print(z.grad_fn)

In [ ]:
w = 2 * z * z
q = w.mean()
q

### Automatický výpočet gradientů

In [ ]:
print(x.grad)

In [ ]:
q.backward()

In [ ]:
print(x.grad)

In [ ]:
print(y.grad)

## Načtení dat

PyTorch integruje nejpopulárnější testovací datasety přímo ve svém API. CIFAR-10 je jedním z nich. Načtení dat je tak velmi jednoduché:

In [ ]:
from torchvision import datasets
from torchvision import transforms

In [ ]:
trainset = datasets.CIFAR10(root='./data', train=True, download=True)
testset = datasets.CIFAR10(root='./data', train=False, download=True)

X_train = trainset.train_data
y_train = np.array(trainset.train_labels, dtype=np.int64)
X_test = testset.test_data
y_test = np.array(testset.test_labels, dtype=np.int64)

print(type(X_train), X_train.shape, X_train.dtype)
print(type(y_train), y_train.shape, y_train.dtype)
print(type(X_test), X_test.shape, X_test.dtype)
print(type(y_test), y_test.shape, y_test.dtype)

classes = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
num_train, num_test = X_train.shape[0], X_train.shape[0]
x_dim = X_train.shape[1] * X_train.shape[2] * X_train.shape[3]

In [ ]:
for i, cls in enumerate(classes):
    cls_ids, = np.where(y_train == i)
    draw_ids = np.random.choice(cls_ids, size=10)
    
    for j, k in enumerate(draw_ids):
        plt.subplot(10, 10, j * 10 + i + 1)
        plt.imshow(X_train[k])
        plt.axis('off')
        if j == 0:
            plt.title(cls)
plt.show()

### Preprocessing



In [ ]:
def preprocess(rgb_batch, resize=None):
    if isinstance(resize, tuple):
        rgb_batch = [cv2.resize(rgb, (32, 32)) for rgb in rgb_batch]
    X = np.array(rgb_batch, dtype=np.float32) / 255.
    m = np.mean(X, axis=(1, 2))
    X -= m[:, None, None, :]
    X = X.reshape(X.shape[0], -1)
    return X

# Klasifikace neuronovými sítěmi v PyTorch

## PyTorch: sekvenční API

PyTorch nabízí dva základní způsoby definování modelů: sekvenční a funkcionální.

In [ ]:
from torch import nn
import torch.nn.functional as F

Základní třídou reprezentující neuronovou síť je `Sequential`.

In [ ]:
fully_con = nn.Sequential(
    nn.Linear(x_dim, 200),
    nn.ReLU(),
    nn.Linear(200, len(classes)),
)

`Linear(x_dim, 200)` znamená, že vrstva bude mít výstup o rozměru 200.

Model má následující strukturu:

In [ ]:
print(fully_con)

## PyTorch: "funkcionální" API

Vyzkoušíme si také funkcionální API, které se liší pouze ve způsobu definice modelu. Úplně stejný model lze funkcionálním API vytvořit následovně:

In [ ]:
class FullyConnected(nn.Module):
    def __init__(self, input_dim=x_dim, output_dim=len(classes)):
        super().__init__()
        
        self.fc1 = nn.Linear(input_dim, 200)
        self.r1 = nn.ReLU()
        self.fc2 = nn.Linear(200, output_dim)
        
        if CUDA:
            self.cuda()
    
    def forward(self, x):
        z = self.fc1(x)
        z = self.r1(z)
        z = self.fc2(z)
        return z

In [ ]:
fully_con = FullyConnected()

In [ ]:
print(fully_con)

In [ ]:
for name, par in fully_con.named_parameters():
    print(name, par.shape, par.numel())

### Inicializace

Inicializace probíhá při definování vrstvy, v tuto chvíli už mají všechny parametry výchozí hodnoty, zde včetně biasů (dříve jsme je nastavovali na nuly).

In [ ]:
fully_con.fc1.weight

In [ ]:
fully_con.fc1.bias

Jediná inicializace tedy bude vynulovat pole, které bude ukládat historii hodnot lossu a accuracy.

In [ ]:
fc_history = []

### Trénování 



Kritérium lze v PyTorch definovat velmi jednoduše.

In [ ]:
loss_func = nn.CrossEntropyLoss()

Podobně jako v Kerasu i v PyTorch lze volit mezi různými metodami optimalizace. Opět prozatím zůstaneme u (Minibatch) Stochastic Gradient Descentu (SGD). Ten je reprezentován třídou `SGD`, která má několik (hyper)parametrů, z nichž nám známá je `lr` (learning rate), neboli velikost kroku. Nejdůležitější je ale zadat první povinný argument, který bude optimizéru říkat, jaké parametry modelu má vlastně updatovat. Zde chceme učit všechny vrstvy modelu, jako seznam tedy zadáme `model.parameters()`.

In [ ]:
from torch import optim

In [ ]:
optimizer = optim.SGD(fully_con.parameters(), lr=0.01)

Zatímco v Kerasu stačí zavolat `model.fit(X, y)`, ve více low-level zaměřeném PyTorch dá trénovací cyklus trochu více práce. Zde je nutné smyčku volající dopředný a zpětný průchod našeho modelu napsat ručně.

In [ ]:
def train(model, loss_func, optimizer, X_data, y_data, prep_fn=None, batch_size=20, history_output=None, num_iters=None,
          print_every=500, permute=True):
    model.train()
    
    if isinstance(history_output, list):
        history_output.append({'loss': [], 'acc': []})
        
    if permute:
        perm = np.random.permutation(X_data.shape[0])
    else:
        perm = np.arange(X_data.shape[0], dtype=np.int)
       
    if num_iters is None:
        num_iters = int(np.ceil(X_data.shape[0] / batch_size))

    for n in range(num_iters):
        # stejne jako minule navzorkujeme data
        batch_ids = perm[n * batch_size:(n + 1) * batch_size]
        x = X_data[batch_ids]
        if prep_fn is not None:
            x = prep_fn(x)
        y = y_data[batch_ids]
        
        x = Variable(torch.from_numpy(x))
        y = Variable(torch.from_numpy(y))
        
        if CUDA:
            x, y = x.cuda(), y.cuda()
        
        # dopredny pruchod
        score = model(x)
        
        # loss
        loss = loss_func(score, y)
        
        # zpetny pruchod = vypocet gradientu; pred samotnym vypoctem je nutne manualne
        # vynulovat gradienty z predchozi iterace, jinak by se vypoctene gradienty pouze
        # pricetly k minulym
        optimizer.zero_grad()
        loss.backward()
        
        # update parametru
        optimizer.step()
        
        # spocitat statistiky
        _, pred = score.data.max(dim=1)
        num_correct = torch.sum(pred == y.data)
        acc = num_correct / x.shape[0]
        
        if isinstance(history_output, list):
            history_output[-1]['loss'].append(float(loss))
            history_output[-1]['acc'].append(acc)
        
        # cas od casu vypis, jak se dari
        if (n + 1) % print_every == 0:
            print('trn {}/{}: loss={:.3f}, acc={:.3f}'.format(n + 1, num_iters, float(loss), acc))

In [ ]:
%%time
train(fully_con, loss_func, optimizer, X_train, y_train, prep_fn=preprocess, history_output=fc_history, print_every=500)

### Validace



In [ ]:
def validate(model, X_data, y_data, prep_fn=None, batch_size=20, history_output=None, print_every=100):
    model.eval()
    
    if isinstance(history_output, list):
        history_output[-1].update({'val_loss': [], 'val_acc': []})
    
    num_iters = int(np.ceil(X_data.shape[0] / batch_size))
    
    for n in range(num_iters):
        batch_ids = np.arange(n * batch_size, (n + 1) * batch_size)
        x = X_data[batch_ids]
        if prep_fn is not None:
            x = prep_fn(x)
        y = y_data[batch_ids]
        
        x = Variable(torch.from_numpy(x))
        y = Variable(torch.from_numpy(y))
        
        if CUDA:
            x, y = x.cuda(), y.cuda()
        
        # dopredny pruchod
        score = model(x)
        
        # loss
        loss = loss_func(score, y)
        
        # spocitat statistiky
        _, pred = score.data.max(dim=1)
        num_correct = torch.sum(pred == y.data)
        acc = num_correct / x.shape[0]
        
        if isinstance(history_output, list):
            history_output[-1]['val_loss'].append(float(loss))
            history_output[-1]['val_acc'].append(acc)
        
        # cas od casu vypis, jak se dari
        if (n + 1) % print_every == 0:
            print('val {}/{}: loss={:.3f}, acc={:.3f}'.format(n + 1, num_iters, float(loss), acc))

In [ ]:
%%time
validate(fully_con, X_test, y_test, prep_fn=preprocess, history_output=fc_history, print_every=100)

In [ ]:
def plot_history(history):
    colors = plt.rcParams['axes.prop_cycle'].by_key()['color']
    
    plt.figure(figsize=(10, 3))
    for mi, metric in enumerate(('loss', 'acc')):
        plt.subplot(1, 2, mi + 1)
        
        last = 0.
        trn_data, val_data = [], []
        for h in history:
            trn_data.append(np.mean(h[metric]))
            val_data.append(np.mean(h['val_' + metric]) if h.get('val_' + metric) else last)
            last = val_data[-1]
        
        plt.plot(trn_data)
        plt.plot(val_data)
        
        plt.ylabel(metric)
    
    plt.show()

In [ ]:
plot_history(fc_history)

## Vše u sebe

In [ ]:
fully_con = FullyConnected()
fc_history = []

In [ ]:
optimizer = optim.SGD(fully_con.parameters(), lr=0.005)
loss_func = nn.CrossEntropyLoss()

In [ ]:
%%time
for epoch in range(10):
    train(fully_con, loss_func, optimizer, X_train, y_train, history_output=fc_history, print_every=500)
    validate(fully_con, X_test, y_test, history_output=fc_history, print_every=100)

In [ ]:
plot_history(fc_history)

### Predikce

Pokud máme natrénovaný model, můžeme predikovat třídu neznámého obrázku metodou `predict`.

In [ ]:
import PIL
import requests

In [ ]:
url = 'https://camo.githubusercontent.com/7e8b7ea66e6dbc2fbcd72bc2a105ed464de1b6b1/687474703a2f2f6661726d352e737461746963666c69636b722e636f6d2f343037302f343731373336333934355f623733616664373861392e6a7067'
img = PIL.Image.open(requests.get(url, stream=True).raw)
rgb_test = np.array(img)

plt.imshow(rgb_test)
plt.show()

In [ ]:
def predict_and_show(model, rgb):
    x = preprocess([rgb], resize=(32, 32))
    x = Variable(torch.from_numpy(x))
    if CUDA:
        x = x.cuda()
    
    score = model(x).data.cpu().numpy().ravel()
    prob = np.exp(score) / np.sum(np.exp(score))
            
    plt.figure(figsize=(5, 5))
    plt.imshow(rgb)
    ids = np.argsort(-score)
    for i, ci in enumerate(ids):
        plt.gcf().text(1., 0.8 - 0.075 * i, '{}: {:.2f} %'.format(classes[ci], 100. * prob[ci]), fontsize=24)
    plt.subplots_adjust()
    plt.show()
    
    return ids[0]

In [ ]:
num_correct = 0
total = 0

In [ ]:
idx = np.random.choice(len(trainset))
ci = predict_and_show(fully_con, X_train[idx])
if ci == y_train[idx]:
    num_correct += 1
total += 1
print(num_correct / total)

In [ ]:
predict_and_show(fully_con, rgb_test)